In [ ]:
# Mounting

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/602/3

In [ ]:
# Importing Libraries

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
file_path = 'apple_watch_data.csv'

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
# Understanding Data

In [ ]:
df.head(5)

In [ ]:
print("Dataset Head:")

In [ ]:
print(df.head())

In [ ]:
print("\nDataset Info:")

In [ ]:
print(df.info())

In [ ]:
print("\nDescriptive Statistics:")

In [ ]:
print(df.describe())

In [ ]:
print("\nMissing Values Initially:")

In [ ]:
print(df.isnull().sum())

In [ ]:
# Cleaning Missing Values

In [ ]:
df = df.rename(columns={

In [ ]:
    'hear_rate': 'heart_rate',

In [ ]:
    'entropy_setps': 'entropy_steps'

In [ ]:
})

In [ ]:
knn_imputer = KNNImputer(n_neighbors=5)

In [ ]:
df[['steps', 'calories', 'distance']] = knn_imputer.fit_transform(df[['steps', 'calories', 'distance']])

In [ ]:
df['heart_rate'] = df['heart_rate'].fillna(method='ffill')

In [ ]:
print("\nMissing Values After Cleaning:")

In [ ]:
print(df.isnull().sum())

In [ ]:
# Handling Outliers and Invalid Entries

In [ ]:
df['heart_rate'] = pd.to_numeric(df['heart_rate'], errors='coerce')

In [ ]:
df.dropna(subset=['heart_rate', 'steps', 'calories', 'distance'], inplace=True)

In [ ]:
scaler = RobustScaler()

In [ ]:
scaled_columns = ['heart_rate', 'steps', 'calories', 'distance']

In [ ]:
df_scaled = scaler.fit_transform(df[scaled_columns])

In [ ]:
df_scaled = pd.DataFrame(df_scaled, columns=scaled_columns)

In [ ]:
iso_forest = IsolationForest(n_estimators=100, contamination=0.05)

In [ ]:
outliers = iso_forest.fit_predict(df_scaled)

In [ ]:
df['outlier'] = outliers

In [ ]:
df_cleaned = df[df['outlier'] != -1]

In [ ]:
# Visualizing Data

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(12, 20))

In [ ]:
axes = axes.flatten()

In [ ]:
for i, col in enumerate(scaled_columns):

In [ ]:
    sns.boxplot(x=df[col], ax=axes[2*i])

In [ ]:
    sns.boxplot(x=df_cleaned[col], ax=axes[2*i + 1])

In [ ]:
plt.tight_layout()

In [ ]:
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))

In [ ]:
sns.histplot(df['heart_rate'].dropna(), kde=True)

In [ ]:
plt.show()

In [ ]:
if df['heart_rate'].isnull().any():

In [ ]:
    model = LinearRegression()

In [ ]:
    train_df = df.dropna(subset=['heart_rate'])

In [ ]:
    X_train = train_df[['age', 'weight', 'steps']]

In [ ]:
    y_train = train_df['heart_rate']

In [ ]:
    model.fit(X_train, y_train)

In [ ]:
    missing_values = df[df['heart_rate'].isnull()]

In [ ]:
    X_missing = missing_values[['age', 'weight', 'steps']]

In [ ]:
    if not X_missing.empty:

In [ ]:
        predicted_values = model.predict(X_missing)

In [ ]:
        df.loc[df['heart_rate'].isnull(), 'heart_rate'] = predicted_values

In [ ]:
plt.figure(figsize=(8, 4))

In [ ]:
sns.boxplot(x=df['heart_rate'])

In [ ]:
plt.show()

In [ ]:
df['heart_rate'] = df['heart_rate'].apply(lambda x: abs(x) if x < 0 else x)

In [ ]:
plt.figure(figsize=(8, 4))

In [ ]:
sns.boxplot(x=df['heart_rate'])

In [ ]:
plt.show()

In [ ]:
# Adjust Activity Labels

In [ ]:
original_activities = df['activity'].copy()

In [ ]:
df['activity'] = df['activity'].replace('Self Pace walk', 'Walking')

In [ ]:
def map_activities(activity):

In [ ]:
    if activity == "Stnding":

In [ ]:
        return np.random.choice(["Lying", "Sitting", "Walking"])

In [ ]:
    return activity

In [ ]:
df['activity'] = df['activity'].apply(map_activities)

In [ ]:
original_freq = original_activities.value_counts()

In [ ]:
corrected_freq = df['activity'].value_counts()

In [ ]:
plt.figure(figsize=(14, 6))

In [ ]:
plt.subplot(1, 2, 1)

In [ ]:
sns.barplot(x=original_freq.index, y=original_freq.values, palette="viridis")

In [ ]:
plt.subplot(1, 2, 2)

In [ ]:
sns.barplot(x=corrected_freq.index, y=corrected_freq.values, palette="viridis")

In [ ]:
plt.tight_layout()

In [ ]:
plt.show()

In [ ]:
df.to_csv('cleaned_data.csv', index=False)